In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [32]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [33]:
X = data.drop(['SalePrice'], axis = 1)
y = data['SalePrice']

Уберем нечисловые данные:

In [34]:
X_num = X.select_dtypes(exclude='object')

In [35]:
X_num.drop(columns=['Id'], inplace=True)

/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Пропуски заменим средними значениями

In [36]:
X_num.isnull().sum()

MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [37]:
X_num = X_num.fillna(X_num.mean())

Нормализуем данные

In [38]:
scaler = StandardScaler()

In [39]:
X_scaled = scaler.fit_transform(X_num)

Построить случайный лес, вывести важность признаков
====

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [41]:
clf_rf = RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)

In [42]:
clf_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_features=0.5, min_samples_leaf=20,
                       n_estimators=10, n_jobs=-1)

In [54]:
imp = pd.Series(clf_rf.feature_importances_)
imp.sort_values(ascending=False)

15    0.125717
11    0.115638
26    0.098358
12    0.083034
2     0.079096
1     0.059211
24    0.054027
3     0.050786
18    0.048499
10    0.039619
8     0.033339
13    0.025797
27    0.025087
5     0.023529
35    0.023212
6     0.019015
4     0.015448
7     0.013525
9     0.013246
28    0.009362
29    0.008928
34    0.007870
31    0.007378
20    0.007277
23    0.007105
22    0.005898
17    0.000000
14    0.000000
16    0.000000
19    0.000000
21    0.000000
25    0.000000
30    0.000000
32    0.000000
33    0.000000
0     0.000000
dtype: float64

Самые важные признаки под номерами : 15, 11, 12

Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
=====

In [44]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [45]:
classifier = StackingClassifier(
    [
        ('tree_clf', DecisionTreeClassifier()),
        ('kn_clf', KNeighborsClassifier()),
        ('log_reg', LogisticRegression())
    ],
LogisticRegression())

In [46]:
classifier.fit(X_scaled, y)

/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:905: RuntimeWarning: Number of classes in training fold (583) does not match total number of classes

/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:905: RuntimeWarning: Number of classes in training fold (587) does not match total number of classes (663). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  RuntimeWarning)


StackingClassifier(estimators=[('tree_clf', DecisionTreeClassifier()),
                               ('kn_clf', KNeighborsClassifier()),
                               ('log_reg', LogisticRegression())],
                   final_estimator=LogisticRegression())

In [47]:
classifier.named_estimators_['log_reg']

LogisticRegression()

In [48]:
y_pred_proba_log_reg = classifier.named_estimators_['log_reg'].predict_proba(X_scaled)

In [49]:
y_pred_proba_kn_clf = classifier.named_estimators_['kn_clf'].predict_proba(X_scaled)

In [50]:
y_pred_proba_tree_clf = classifier.named_estimators_['tree_clf'].predict_proba(X_scaled)

In [51]:
y_pred_proba = classifier.predict_proba(X_scaled)

In [53]:
classifier.final_estimator_

LogisticRegression()